# AI DDE Hackathon

Update the cell below with your Huggingface token (see: https://huggingface.co/docs/hub/en/security-tokens) and ensure you have permssion to use the LLama 3 Model (https://huggingface.co/meta-llama/Llama-3.1-8B). 

In [ ]:
# Install required packages
!pip install markitdown langchain chromadb gradio spacy
!pip install flash-attn git+https://github.com/huggingface/transformers.git triton


! pip install pdfplumber pymupdf
# Import basic libraries
import os
from markitdown import MarkItDown
from transformers import AutoTokenizer, AutoModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
import logging
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

HF_TOKEN = "YOUR_TOKEN"

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')
# Create documents folder
DOCUMENTS_PATH = '/content/drive/MyDrive/legal_documents'
if not os.path.exists(DOCUMENTS_PATH):
    os.makedirs(DOCUMENTS_PATH)

Document Processor Class

This cell defines our core document processing system:

In [ ]:
import pdfplumber
import re
import fitz  # PyMuPDF

class PDFProccessor:

  pdf_path = None

  def __init__(self, pdf_path):
    self.pdf_path = pdf_path
    pass

  def process_pdf(self):
    text = self.open_pdf()
    print(self.remove_page_numbers(text))

  def open_pdf(self):
    with pdfplumber.open(self.pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

  def remove_page_numbers(self, text):
    extrapolations_pattern =  r"^\s*THE\s+EXTRAPOLATIONS\s+DOCUMENT(?:\s*\n\s*|\s+)*\d+\s*$"
    dangling_page_number = r"^\s*\d+\s*$"
    clean_text = re.sub(extrapolations_pattern, "", text, flags=re.MULTILINE)
    return re.sub(dangling_page_number, "", clean_text, flags=re.MULTILINE)


This I think might help reduce the tokens for RAG.

In [ ]:
import re
import pdfplumber

def extract_chunks_after_toc(pdf_path, start_after="I. Introduction"):
    content_to_process = ""
    start_processing = False

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()

            if not start_processing:
                # Detect and skip the Table of Contents
                if "Table of Contents" in text:
                    continue

                # Look for the start point after TOC
                match = re.search(rf"{re.escape(start_after)}", text, re.IGNORECASE)
                if match:
                    # Start processing text after the specified section
                    _, remaining_text = re.split(rf"{re.escape(start_after)}", text, flags=re.IGNORECASE, maxsplit=1)
                    content_to_process += remaining_text.strip()
                    start_processing = True
            else:
                # Append remaining text from the document
                content_to_process += "\n" + text.strip()

    # Chunk content further (if needed)
    split_pattern = r"(?=\n[A-Z]\.\s)|(?=\n[A-Z]\.\d+\s)"  # Match sections A., A.1, etc.
    chunks = re.split(split_pattern, content_to_process)
    cleaned_chunks = [chunk.strip() for chunk in chunks if chunk.strip()]  # Remove empty chunks

    return cleaned_chunks

pdf_path = "/content/drive/MyDrive/legal_documents/Leiden Guidelines on the Use of DDE in ICCTs_20220404.pdf"
# chunks = extract_chunks_after_section(pdf_path)
# for i, chunk in enumerate(chunks):
#     print(f"Chunk \n{i+1}:\n{chunk}\n{'-'*50}")


chunks = extract_chunks_after_toc(pdf_path)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n{'-' * 50}")


def extract_headers_and_chunks(chunks):
    headers_with_chunks = []

    for chunk in chunks:
        # Match header at the start of the chunk (e.g., C. Aerial and Satellite Images)
        match = re.match(r"^[A-Z](?:\.\d+)?\.\s.+", chunk)
        if match:
            header = match.group(0).strip()  # Extract the header
            content = chunk[len(header):].strip()  # Remaining content after the header
            headers_with_chunks.append({"header": header, "content": content})
        else:
            headers_with_chunks.append({"header": "Unknown Header", "content": chunk.strip()})

    return headers_with_chunks


Chunk 1:
Digitally Derived Evidence (DDE) is increasingly used in international criminal courts and
tribunals to prosecute perpetrators of international crimes. Advanced digital tools, including
aerial photography, mobile devices, video, intercepted communications, amongst others,
capture new and vast quantities of data, which can add supplementary and supporting data to
existing evidence. For example, while an eyewitness account may provide relevant information
regarding an event, a satellite image may unearth information that would otherwise be
inaccessible. Furthermore, phone and computer records may provide data relevant to an
individual’s activities, or a video may be geo-located, allowing investigators to explore
additional details that a witness may have forgotten.1 Given the proliferation of digitally
derived evidence and increasing reliance upon it for prosecutions, there is every possibility that
digital evidence may become the primary evidence upon which some convictions are

In [ ]:
headers_with_chunks = extract_headers_and_chunks(chunks)

for item in headers_with_chunks:
    print(f"Section Header: {item['header']}\n")


Section Header: Unknown Header

Section Header: A. Definition of DDE

Section Header: B. Methodology

Section Header: C. Structure of the Leiden Guidelines

Section Header: D. Scope of the Leiden Guidelines

Section Header: A. Videos

Section Header: B. Photographs

Section Header: C. Aerial and Satellite Images

Section Header: D. Intercepts

Section Header: E. Call Data Records

Section Header: F. Audio Recordings



In [ ]:
import chromadb

# vecorisation could be improved

class DocumentProcessor:
    def __init__(self):
        """Initialize the document processor with necessary components."""
        # Set up embedding model
        self.tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
        self.model.eval()
        # Initialize document converter
        self.md = MarkItDown()
        # Set up vector database
        self.vector_db = chromadb.Client()
        self.collection = self.vector_db.get_or_create_collection(name="legal_docs")

    def process_document(self, file_path):
        """Convert document to text and generate embeddings."""
        try:
            pdf_processor = PDFProccessor(file_path)
            # Convert document to text
            conversion_result = self.md.convert(file_path)
            conversion_result_text = self.md.convert(file_path).text_content
            # TODO - teh leiden guidelines contain a section of keywwords for each section - these should be parsed out and each section should be stored seperately
            conversion_result_text = pdf_processor.remove_page_numbers(conversion_result_text)
            print(conversion_result)

            # Create embeddings
            inputs = self.tokenizer(
                conversion_result_text,
                return_tensors="pt",
                truncation=True
            )
            # Use GPU if available
            if torch.cuda.is_available():
                self.model.to('cuda')
                inputs = {k: v.to('cuda') for k, v in inputs.items()}
            # Generate embeddings
            with torch.no_grad():
                outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy().tolist()
            # **************
            return {
                'text': conversion_result_text,
                'embeddings': embeddings,
                'metadata': {}
            }
            # Note: we don't seem to get metadata from the docs anyway so better manually adding
            getattr(conversion_result, 'metadata', {})
        except Exception as e:
            logger.error(f"Error processing document {file_path}: {str(e)}")
            raise

    def store_document(self, doc_id, text, embedding, metadata=None):
        """Store document in the vector database."""
        if metadata is None:
            metadata = {}
        self.collection.add(
            documents=[text],
            embeddings=[embedding],
            metadatas=[metadata],
            ids=[doc_id]
        )

    def find_relevant_documents(self, query, metadata_titles, metadata_keywords, n_results=3):
        """Find relevant documents for a given query."""

        filter_dict = {}

        if metadata_titles and len(metadata_titles) > 0:
          filter_dict["title"] = {"$in": metadata_titles}

        if metadata_keywords:
          filter_dict["keywords"] = {"$in": metadata_keywords}

        filter = None if len(filter_dict) == 0 else filter_dict

        results = self.collection.query(
            query_texts=[query],
            where=filter,
            n_results=n_results
        )
        return [
            {
                'text': doc_text,
                'id': results['ids'][0][i],
                'metadata': results['metadatas'][0][i]
            }
            for i, doc_text in enumerate(results['documents'][0])
        ]
# Initialize processor
processor = DocumentProcessor()
# Move to GPU if available
if torch.cuda.is_available():
    processor.model = processor.model.to('cuda')

Set Up LLaMA Model

Go to HuggingFace and search for the llama model you want to use. For example, 3.1. Request permission to use it and get a HuggingFace token.

This cell initializes the LLaMA model for generating responses:

In [ ]:
# fine tune llama to make this better
# experiment with prompts
# open ai option
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B",
    token=HF_TOKEN
)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B",
    token=HF_TOKEN
)

Process Documents

This cell processes all documents in your legal_documents folder:

In [ ]:

metadata_list = {
    'doc_0_Leiden Guidelines on the Use of DDE in ICCTs_20220404.pdf' : {
      'title': 'Leiden Guidelines on the Use of Digitally Derived Evidence in International Criminal Courts and Tribunals',
      'author': 'Unknown',
      'section-header': '',
      'category': '',
      'keywords': '',
    },
    'doc_1_Extrapolations from Case Law on DDE in ICCTs_20220405.pdf': {
      'title': 'Extrapolations from case law on the use of digitally derived evidence (dde) before international criminal courts and tribunals',
      'author': 'Unknown',
      'section-header': '',
      'category': '',
      'keywords': '',
      },
    'doc_2_Case Summaries-The Use of DDE before ICCTs.pdf': {
      'title': 'Analysis of Digitally Derived Evidence from the Jurisprudence of International Tribunals: Cases from the ICC, ICTR, ICTY, IRMCT, SCSL and STL',
      'author': 'Unknown',
      'section-header': '',
      'category': '',
      'keywords': '',
      },
    'doc_3_Fact-Finding-Missions.pdf': {
      'title': 'REPORT ON DIGITALLY DERIVED EVIDENCE USED IN UN HUMAN RIGHTS FACT-FINDING MISSIONS APPROACHES AND STANDARDS OF PROOF',
      'author': 'Unknown',
      'section-header': '',
      'category': '',
      'keywords': '',

      },
    'doc_4_DDE in ICL.pdf': {
      'title': 'REPORT ON DIGITALLY DERIVED EVIDENCE IN INTERNATIONAL CRIMINAL LAW',
      'author': 'Unknown',
      'section-header': '',
      'category': '',
      'keywords': '',
      },
}
# TODO 'section-header': '' each PDF needs chunked and the section header of each chunk need added to the metadata
print(metadata_list['doc_0_Leiden Guidelines on the Use of DDE in ICCTs_20220404.pdf'])

{'title': 'Leiden Guidelines on the Use of Digitally Derived Evidence in International Criminal Courts and Tribunals', 'author': 'Unknown', 'section-header': '', 'category': '', 'keywords': ''}


In [ ]:
# Get list of documents
# improves the inteface
# disclaimers - agent answers - next agent makes it accesible - next agent is a lawyer that critiques answer

document_files = [
    f for f in os.listdir(DOCUMENTS_PATH)
    if f.endswith(('.pdf', '.docx', '.txt', '.html', '.pptx'))
]
if not document_files:
    print("⚠️ No documents found! Add some to your legal_documents folder")
else:
    print(f"Found {len(document_files)} documents to process")
    for idx, document in enumerate(document_files):
        print(f"Processing {document}...")
        file_path = os.path.join(DOCUMENTS_PATH, document)
        # Process document
        result = processor.process_document(file_path)
        # Store in database
        doc_id = f"doc_{idx}_{document}"
        metadata = metadata_list[doc_id]
        processor.store_document(
            doc_id=doc_id,
            text=result['text'],
            embedding=result['embeddings'],
            metadata=metadata
        )
        print(f"✅ Finished storing {document} in Chroma\n")
        # except Exception as e:
        #     print(f"Error processing {document}: {str(e)}")

Found 5 documents to process
Processing Leiden Guidelines on the Use of DDE in ICCTs_20220404.pdf...
✅ Finished storing Leiden Guidelines on the Use of DDE in ICCTs_20220404.pdf in Chroma

Processing Extrapolations from Case Law on DDE in ICCTs_20220405.pdf...
✅ Finished storing Extrapolations from Case Law on DDE in ICCTs_20220405.pdf in Chroma

Processing Case Summaries-The Use of DDE before ICCTs.pdf...
✅ Finished storing Case Summaries-The Use of DDE before ICCTs.pdf in Chroma

Processing Fact-Finding-Missions.pdf...
✅ Finished storing Fact-Finding-Missions.pdf in Chroma

Processing DDE in ICL.pdf...
✅ Finished storing DDE in ICL.pdf in Chroma



Question-Answering Function

This cell defines the function that generates answers using LLaMA. You may alter the values if you know what you’re doing :)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
# Sample function for extracting keywords
def extract_keywords(text):
    doc = nlp(text)
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return keywords


In [ ]:
!pip install fuzzywuzzy

/usr/local/lib/python3.11/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/fuzzywuzzy-0.18.0.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


In [ ]:
# --- Document Excerpts ---
# {truncated_context}
# Function to enhance the query by adding the extracted keywords
from fuzzywuzzy import fuzz

titles_list = [doc_metadata['title'] for doc_metadata in metadata_list.values()]

def find_full_title(query, titles_list, threshold=60):
    matches = [title for title in titles_list if fuzz.partial_ratio(query.lower(), title.lower()) >= threshold]
    return matches

keywords_list = [doc_metadata['keywords'] for doc_metadata in metadata_list.values()]

def find_keywords(search_terms, keywords):
    for search_term in search_terms:
        if search_term.lower() in keywords:
            return search_term.lower()
    return None  # No matching title found

# TODO - if the chat bot doesn't recognise any keywords it should prompt back and say something like:
#  'Ask me about digital evidence related to photographic, video or etc. evidence.'
def ask_question_llama(question, leiden_guide_lines, case_law):
    # always Leiden Guidelines
    metadata_titles = [titles_list[0]]
    if case_law:
      # Extrapolations from case law
      metadata_titles.append(titles_list[1])
    # if
      # Cases from the ICC, ICTR, ICTY, IRMCT, SCSL and STL
    # if
      # UN HUMAN RIGHTS FACT-FINDING
    # if
      # INTERNATIONAL CRIMINAL LAW
    extracted_keywords = extract_keywords(question)

    metadata_keywords = find_keywords(extracted_keywords, keywords_list)
    # metadata_titles = find_full_title(question, titles_list)
    """Generate an answer to a legal question using LLaMA."""
    # Get relevant documents
    relevant_docs = processor.find_relevant_documents(query=question, metadata_keywords=metadata_keywords, metadata_titles=metadata_titles, n_results=5)

    # Prepare context
    context_pieces = [doc['text'][4000:5000] for doc in relevant_docs]
    titles = [doc['metadata']['title'] for doc in relevant_docs]
    keywords = [doc['metadata']['keywords'] for doc in relevant_docs]
    truncated_context = "\n".join(context_pieces)
    # Create prompt

    full_prompt = f"""You are a Human Rights Lawyer using the documents below to answer the following question.
--- Question ---
{question}

Based on the documents above, provide a clear, concise answer. If relevant, refer to legal precedent, case law, or any specific details from the documents. Do not simply restate the question; make sure the answer is grounded in the provided content.


Answer:
"""

    # Prepare for generation
    if torch.cuda.is_available():
        model.to('cuda')
    # TODO - increase the token limit to allow for more text
    inputs = tokenizer(
        full_prompt,
        return_tensors="pt",
        max_length=1024,
        truncation=True
    )
    if torch.cuda.is_available():
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
    # TODO - increase the token limit to allow for more text
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=800,
            temperature=0.7,
            do_sample=True
        )
    # Process output
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Answer:" in raw_output:
        final_answer = raw_output.split("Answer:", 1)[1].strip()
    else:
        final_answer = raw_output

    docs = ",\n - ".join(metadata_titles)
    final_answer = final_answer + f'\n\n DOCUMENTS: {docs}' + '\n\n NOTE: this is only guidance based on past case law.'
    # print(final_answer )
    return final_answer

    #

answer = ask_question_llama("Are you using the Leiden Guidelines and case law ?", True, True)
print(answer)

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 85.7MiB/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The documents provided do not explicitly state whether the lawyer is using the Leiden Guidelines and case law. However, based on the information provided, it can be inferred that the lawyer is likely familiar with these resources. The first document, titled "Leiden Principles," outlines a set of guidelines for lawyers working on human rights cases. It emphasizes the importance of respecting cultural differences and ensuring that legal representation is accessible to all individuals, regardless of their socioeconomic status or geographic location. This aligns closely with the principles outlined in the Leiden Guidelines, which focus on ensuring that legal representation is fair and equitable for all parties involved. Additionally, the second document, which is a court ruling, references the use of case law as a source of legal precedent. This suggests that the lawyer may be familiar with the concept of relying on previous rulings to inform their own arguments and interpretations of the 

Create User Interface

Finally, we can also create the Gradio interface:

In [ ]:
import gradio as gr

def create_interface():
    demo = gr.Interface(
        fn=ask_question_llama,
        inputs=[
            gr.Textbox(
                label="Your Legal Question",
                placeholder="Ask any question about your legal documents...",
                lines=3
            ),
            gr.Checkbox(label="Open Source Investigator", value=True),
            gr.Checkbox(label="Lawyer", value=False),
            ],
        outputs=[
            gr.Markdown(
                label="Answer",
            )
        ],

        title="Legal Document Assistant",
        description="This AI assistant can answer questions about your legal documents."
    )
    return demo
# Launch interface
demo = create_interface()
demo.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=108, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c815b5b4678a3241ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



---

Everything below here from the original Medium Article

---

Option A: Basic Processing (Free, No API Key Needed)

If you want to use the basic document processor (good for text-based documents), create a new cell and run this code:

In [ ]:
# # Set up basic document processing
# processor = DocumentProcessor()
# # Add basic document conversion
# processor.md = MarkItDown()
# logger.info("✅ Basic document processor ready!")

Option B: Enhanced Processing (Requires OpenAI API Key)

If you want enhanced processing with better image handling and understanding, create a new cell and run this code instead:

In [ ]:
# # First, set your OpenAI key
# OPENAI_API_KEY = "your-openai-key-here"  # Replace with your actual key

# # Set up enhanced document processing
# from openai import OpenAI
# processor = DocumentProcessor()
# client = OpenAI(api_key=OPENAI_API_KEY)
# processor.md = MarkItDown(llm_client=client, llm_model="gpt-4")
# logger.info("✅ Enhanced document processor ready!")

Part 3: Choose Your Storage System

Next, decide how you want to store your processed documents. You have two options:
Option A: Local Storage (Free, Good for Small Projects)

If you want to use local storage, create a new cell and run this code:

In [ ]:
# Set up local storage with Chroma
import chromadb
processor.vector_db = chromadb.Client()
logger.info("✅ Local storage ready!")

Option B: Cloud Storage (Requires Qdrant Account)

If you want cloud storage, you need to create an account with Qdrant. Follow their instructions. Then, create a new cell and run this code instead:

In [ ]:
# First, set your Qdrant credentials
QDRANT_URL = "your-qdrant-url-here"      # Replace with your URL
QDRANT_API_KEY = "your-qdrant-key-here"  # Replace with your key
# Set up cloud storage with Qdrant
from qdrant_client import QdrantClient
processor.vector_db = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
logger.info("✅ Cloud storage ready!")

Part 4: Connect to Your Documents

Now you’ll connect to your documents. Everyone should use Google Drive for this part:

In [ ]:
from google.colab import drive
# Mount Google Drive
logger.info("Connecting to Google Drive...")
drive.mount('/content/drive')
# Set up documents folder
DOCUMENTS_PATH = '/content/drive/MyDrive/legal_documents'
if not os.path.exists(DOCUMENTS_PATH):
    os.makedirs(DOCUMENTS_PATH)
    logger.info("Created 'legal_documents' folder in your Google Drive")
    logger.info("⚠️ Please add your documents to this folder before continuing")
else:
    logger.info("✅ Found 'legal_documents' folder")


Mounted at /content/drive


Part 5: Choose Your AI Model

Finally, decide which AI model you want to use for answering questions. You have two options:

Open AI

In [ ]:
# First, ensure you have your OpenAI key
OPENAI_API_KEY = "your-openai-key-here"  # Replace with your actual key
# Set up GPT-4
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
model = ChatOpenAI(
    model="gpt-4",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.7
)
logger.info("✅ GPT-4 model ready!")

Llama

In [ ]:
# First, set your HuggingFace token
HF_TOKEN = "your-huggingface-token-here"  # Replace with your actual token
# Set up Llama
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B",
    token=HF_TOKEN
)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B",
    token=HF_TOKEN
)
logger.info("✅ Llama model ready!")

Part 6: Process Your Documents

Now that everything is set up, create a new cell to process your documents:

In [ ]:
# Get list of documents
document_files = [f for f in os.listdir(DOCUMENTS_PATH)
                 if f.endswith(('.pdf', '.docx', '.txt', '.html', '.pptx'))]

if not document_files:
    logger.info("⚠️ No documents found! Add some to your legal_documents folder")
else:
    logger.info(f"Found {len(document_files)} documents to process")
    # Process each document
    for document in document_files:
        try:
            logger.info(f"Processing {document}...")
            file_path = os.path.join(DOCUMENTS_PATH, document)
            result = processor.process_document(file_path)
            logger.info(f"✅ Processed {document}")
        except Exception as e:
            logger.error(f"Error processing {document}: {str(e)}")

Part 7: Final Testing

Create one last cell to test your assistant. The code depends on which AI model you chose:
If You Chose GPT-4:

In [ ]:
def ask_question(question):
    prompt = f"Based on the legal documents, please answer: {question}"
    return model.predict(prompt)

# Test the system
test_question = "What are the main terms of the agreement?"
answer = ask_question(test_question)
print(f"Question: {test_question}")
print(f"Answer: {answer}")

If You Chose Llama:

In [ ]:
def ask_question(question):
    prompt = f"Question about legal documents: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the system
test_question = "What are the main terms of the agreement?"
answer = ask_question(test_question)
print(f"Question: {test_question}")
print(f"Answer: {answer}")

Part 8: Creating Your User Interface

Now that we have our assistant working, let’s create a user-friendly interface. This will make it easy for anyone to ask questions about your legal documents. We’ll use Gradio, a tool that helps create web interfaces for AI applications.

In [ ]:
# Install Gradio for our interface
!pip install gradio
print("✅ Gradio installation complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 65.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/MarkupSafe-2.1.5.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/semantic_version-2.10.0.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


✅ Gradio installation complete!


Step 2: Choose Your Interface Setup

Now you’ll need to set up the interface based on which AI model you chose earlier. Pick the option that matches your previous choice:
Option A: Interface with GPT-4

If you’re using GPT-4, create a new cell and run this code:

In [ ]:
import gradio as gr
from typing import List, Dict
import logging
def create_gpt4_interface(model, processor):
    """Creates an interface for GPT-4 based assistant"""
    def get_response(question: str) -> str:
        """Process a question and get GPT-4's response"""
        try:
            # Find relevant document sections
            relevant_docs = processor.find_relevant_documents(question)
            context = "\n\n".join(doc.text for doc in relevant_docs)
            # Create our prompt
            prompt = f"""As a legal expert, please answer this question based on
            the provided documents:
            Documents: {context}
            Question: {question}
            """
            # Get GPT-4's response
            response = model.predict(prompt)
            return response
        except Exception as e:
            logging.error(f"Error processing question: {str(e)}")
            return f"I encountered an error: {str(e)}"
    # Create the interface
    demo = gr.Interface(
        fn=get_response,
        inputs=[
            gr.Textbox(
                label="Your Legal Question",
                placeholder="Ask any question about your legal documents...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(
                label="Answer",
                lines=10
            )
        ],
        title="Legal Document Assistant (GPT-4)",
        description="""This AI assistant can answer questions about your legal documents.
        It uses GPT-4 to provide accurate, contextual responses based on your documents."""
    )
    return demo
# Create and launch the interface
demo = create_gpt4_interface(model, processor)
demo.launch(share=True)  # share=True creates a public link you can share
print("✅ Interface is ready! Click the link above to start using your assistant.")

/usr/local/lib/python3.10/dist-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to 

NameError: name 'model' is not defined

Option B: Interface with Llama

If you’re using Llama, create a new cell and run this code instead:

In [ ]:
import gradio as gr
from typing import List, Dict
import logging

def create_llama_interface(model, tokenizer, processor):
    """Creates an interface for Llama-based assistant"""
    def get_response(question: str) -> str:
        """Process a question and get Llama's response"""
        try:
            # Find relevant document sections
            relevant_docs = processor.find_relevant_documents(question)
            context = "\n\n".join(doc.text for doc in relevant_docs)
            # Create our prompt
            prompt = f"""Please answer this legal question based on the provided
            documents. Be specific and cite relevant sections.
            Documents: {context}
            Question: {question}
            Answer:"""
            # Prepare for Llama
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                max_length=2048,
                truncation=True
            )
            # Generate response
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.7,
                do_sample=True
            )
            # Decode response
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            return response.split("Answer:")[-1].strip()
        except Exception as e:
            logging.error(f"Error processing question: {str(e)}")
            return f"I encountered an error: {str(e)}"
    # Create the interface
    demo = gr.Interface(
        fn=get_response,
        inputs=[
            gr.Textbox(
                label="Your Legal Question",
                placeholder="Ask any question about your legal documents...",
                lines=3
            )
        ],
        outputs=[
            gr.Textbox(
                label="Answer",
                lines=10
            )
        ],
        title="Legal Document Assistant (Llama)",
        description="""This AI assistant can answer questions about your legal documents.
        It uses Llama to provide detailed responses based on your documents."""
    )
    return demo
# Create and launch the interface
demo = create_llama_interface(model, tokenizer, processor)
demo.launch(share=True)  # share=True creates a public link you can share
print("✅ Interface is ready! Click the link above to start using your assistant.")

Step 3: Enhanced Interface (Optional)

If you want a more sophisticated interface with additional features, create a new cell and run this code (works with either model):

In [ ]:
import gradio as gr
from typing import List, Dict
import logging

def create_advanced_interface(model, processor, model_type="gpt-4"):
    """Creates an enhanced interface with additional features"""
    def process_query(
        question: str,
        show_sources: bool,
        response_length: str
    ) -> Dict[str, str]:
        """Process a question with additional options"""
        try:
            # Find relevant documents
            relevant_docs = processor.find_relevant_documents(question)
            context = "\n\n".join(doc.text for doc in relevant_docs)
            # Adjust response length
            max_tokens = {
                "Brief": 100,
                "Detailed": 300,
                "Comprehensive": 500
            }[response_length]
            # Get response based on model type
            if model_type == "gpt-4":
                response = model.predict(
                    f"Please provide a {response_length.lower()} answer: " + context
                )
            else:  # Llama
                inputs = tokenizer(context, return_tensors="pt", truncation=True)
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=max_tokens,
                    temperature=0.7
                )
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            # Prepare return value
            result = {"answer": response}
            # Add sources if requested
            if show_sources:
                sources = "\n\n".join(
                    f"From {doc.source}:\n{doc.text[:200]}..."
                    for doc in relevant_docs
                )
                result["sources"] = sources
            return result
        except Exception as e:
            logging.error(f"Error: {str(e)}")
            return {"answer": f"Error: {str(e)}", "sources": ""}
    # Create enhanced interface
    demo = gr.Interface(
        fn=process_query,
        inputs=[
            gr.Textbox(
                label="Your Legal Question",
                placeholder="Ask any question about your legal documents...",
                lines=3
            ),
            gr.Checkbox(label="Show Source Documents"),
            gr.Radio(
                choices=["Brief", "Detailed", "Comprehensive"],
                label="Response Length",
                value="Detailed"
            )
        ],
        outputs=[
            gr.Textbox(label="Answer", lines=10),
            gr.Textbox(label="Source Documents", visible=False) # Shows when sources requested
        ],
        title=f"Advanced Legal Assistant ({model_type})",
        description="""This enhanced AI assistant can answer questions about your legal documents.
        You can customize the response length and choose to see source documents."""
    )
    return demo
# Create and launch the enhanced interface
demo = create_advanced_interface(model, processor, model_type="gpt-4")  # or "llama"
demo.launch(share=True)
print("✅ Enhanced interface is ready! Click the link above to start using your assistant.")

SyntaxError: invalid syntax (<ipython-input-10-7b47c5f5b820>, line 1)

Interface Features Explained

The interfaces we’ve created offer different features:

    Basic Interface (Options A and B):

    Simple question input
    Clear answer output
    Model-specific optimizations
    Automatic error handling

2. Enhanced Interface (Optional):

    Adjustable response length
    Option to show source documents
    More sophisticated error handling
    Better context management

When using your interface:

    The URL provided will work as long as your Colab notebook is running
    You can share the link with others (if you used share=True)
    The interface will work with your processed documents
    Responses might take a few seconds, especially with longer questions

Troubleshooting Interface Issues

If you encounter problems:

    Interface Won’t Load:

    Make sure all previous cells ran successfully
    Check that Gradio installed correctly
    Verify your model connection is working

2. Slow Responses:

    Try shorter questions first
    Reduce the context window size
    Use the “Brief” response length option

3. Error Messages:

    Check your API keys are still valid
    Verify your documents were processed correctly
    Look for error messages in the Colab output

Remember:

    Keep your Colab notebook running while using the interface
    The public URL changes each time you run the cell
    Save your interface URL if you want to share it
    Monitor your API usage if using GPT-4

Optional Advanced Step: Teaching Your Assistant

If you’ve been using your legal assistant and want to make it even better at handling your specific legal questions, you can teach it using your own examples. This process has two main parts:

    Creating training examples (easier)
    Fine-tuning and deploying the model (more advanced)

You can do just the first part to prepare your training data, and come back to the second part when you’re ready for the more technical steps.
Part 1: Creating Your Training Examples

First, let’s create some examples that will help your assistant learn. We’ll use a friendly tool that makes this process easier. Create a new notebook called “Legal_Assistant_Training” and add this cell:

In [ ]:
import json
import os
from IPython.display import display, clear_output
import ipywidgets as widgets

class ExampleCreator:
    """A friendly tool to help create training examples"""
    def __init__(self):
        self.examples = []
        self.filename = 'training_data.jsonl'
        # Create our input boxes
        self.question_box = widgets.Textarea(
            description='Question:',
            placeholder='Type your legal question here...',
            layout={'width': '90%', 'height': '100px'}
        )
        self.answer_box = widgets.Textarea(
            description='Answer:',
            placeholder='Type the correct answer here...',
            layout={'width': '90%', 'height': '200px'}
        )
        # Create our buttons
        self.save_button = widgets.Button(description='Save Example')
        self.save_button.on_click(self.save_example)
        self.display_button = widgets.Button(description='Show All Examples')
        self.display_button.on_click(self.show_examples)
        # Show our tool
        display(self.question_box)
        display(self.answer_box)
        display(self.save_button)
        display(self.display_button)
    def save_example(self, button):
        """Save a new example"""
        question = self.question_box.value.strip()
        answer = self.answer_box.value.strip()
        if not question or not answer:
            print("❌ Please provide both a question and an answer!")
            return
        # Create the example
        example = {
            "conversations": [
                {"from": "human", "value": question},
                {"from": "gpt", "value": answer}
            ]
        }
        # Save it
        with open(self.filename, 'a') as f:
            f.write(json.dumps(example) + '\n')
        self.examples.append(example)
        # Clear the boxes for next example
        self.question_box.value = ''
        self.answer_box.value = ''
        print(f"✅ Example saved! You now have {len(self.examples)} examples.")
    def show_examples(self, button):
        """Show all saved examples"""
        clear_output(wait=True)
        print(f"Your {len(self.examples)} Training Examples:\n")
        for i, example in enumerate(self.examples, 1):
            print(f"Example {i}:")
            print(f"Q: {example['conversations'][0]['value']}")
            print(f"A: {example['conversations'][1]['value']}\n")
        # Show our tool again
        display(self.question_box)
        display(self.answer_box)
        display(self.save_button)
        display(self.display_button)
# Create our example creator tool
creator = ExampleCreator()
print("✨ Example Creator is ready! Start adding your training examples above.")

Let’s add some starter examples to help you understand the format. Add this cell:

In [ ]:
# Some example legal questions and answers to get you started
starter_examples = [
    {
        "question": "What makes a contract valid?",
        "answer": "A valid contract requires four essential elements: 1) Offer and acceptance, "
                 "2) Consideration (something of value exchanged), 3) Intention to create "
                 "legal relations, and 4) Capacity of the parties to contract. The agreement "
                 "must also be legal and sufficiently certain in its terms."
    },
    {
        "question": "How does force majeure work in contracts?",
        "answer": "Force majeure clauses excuse a party from performing their contractual "
                 "obligations when extraordinary events beyond their control prevent "
                 "performance. These events typically include natural disasters, wars, or "
                 "government actions. The clause must specifically define what constitutes "
                 "force majeure, and the party claiming it must prove the event's impact."
    }
]

# Add these examples using our tool
for example in starter_examples:
    with open('training_data.jsonl', 'a') as f:
        formatted_example = {
            "conversations": [
                {"from": "human", "value": example["question"]},
                {"from": "gpt", "value": example["answer"]}
            ]
        }
        f.write(json.dumps(formatted_example) + '\n')
print("✅ Added starter examples to your training data!")

Now you can use the tool to add your own examples. Here are some tips for creating good examples:

    Include different types of questions:

    What is… (definitions)
    How does… (processes)
    Why is… (reasoning)
    When should… (timing)

2. Make sure your answers:

    Start with a clear main point
    Include specific details
    Use proper legal terminology
    Stay concise but complete

Try to create at least 20–30 examples before moving on to the next part.
## Part 2: Fine-Tuning and Deploying Your Model

Once you have your training examples ready, you can use them to improve your model. This part is more technical and requires a new Colab notebook with more computational resources.

Create a new notebook called “Legal_Assistant_Finetuning” and add these cells:
Cell 1: Install Training Tools

In [ ]:
# Install specialized training packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes datasets

print("✅ Training packages installed")

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4tr4nd6i/unsloth_e4b5dcea890c4a99aa516d0b5d61c0ac
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4tr4nd6i/unsloth_e4b5dcea890c4a99aa516d0b5d61c0ac
  Resolved https://github.com/unslothai/unsloth.git to commit 5dddf27f3ba94506c48251e907031039eecd40d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 8.6 MB/s eta 0:00:00
   ━━━━

/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/bitsandbytes-0.45.0.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/cut_cross_entropy-25.1.1.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/datasets-3.2.0.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/dill-0.3.8.dist-info/top_level.txt' mode='r' encod

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.13.0
    Uninstalling trl-0.13.0:
      Successfully uninstalled trl-0.13.0
✅ Training packages installed


/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/trl-0.8.6.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.10/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.10/dist-packages/xformers-0.0.26.post1.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


Cell 2: Initialize Training

In [ ]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
import torch

# Set up model for training
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None
)
# Prepare for fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    use_gradient_checkpointing=True
)
print("✅ Model prepared for training")

Cell 3: Upload and Train

First, upload your training_data.jsonl file that you created in the previous part of this guide, then run:

In [ ]:
# Load your training data
dataset = load_dataset('json', data_files='training_data.jsonl')

# Configure training
training_args = TrainingArguments(
    output_dir="./legal_assistant_model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch"
)
# Create trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)
# Start training
print("Starting training...")
trainer.train()
print("✅ Training complete!")

Cell 4: Save and Share Your Model

In [ ]:
# First, log in to Hugging Face
from huggingface_hub import login
login()  # You'll need to enter your Hugging Face token

# Save locally
trainer.save_model("./FineTunedLegal")
# Push to Hugging Face Hub
model_name = "your-username/LegalAssistant"  # Change this to your desired name
model.push_to_hub(model_name, tokenizer)
print(f"✅ Model saved and pushed to {model_name}")

Using Your Improved Model

To use your newly trained model, go back to your original legal assistant notebook and update the model loading cell with:

In [ ]:
# Replace the original model loading code with:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "your-username/LegalAssistant",  # Use your model's name
    token="your-huggingface-token"
)
tokenizer = AutoTokenizer.from_pretrained(
    "your-username/LegalAssistant",
    token="your-huggingface-token"
)

Your assistant will now use your improved model that’s been trained on your specific examples. It should be better at handling the types of questions you included in your training data.